cabal update
cabal install --lib prettyprinter

In [1]:
import Data.String
import Data.List (nub,sort) 
import Prettyprinter

---------------  Asociation  --------------------------

type Assoc k v  = [(k,v)] 

find :: Eq k=> k -> Assoc k v -> v
find k t = head [ v |  (k',v)<-t , k==k'] 

type Subst = Assoc String TimeFrame    

--------------- Time Frame --------------------------

data TimeFrame  = TimeFrame Int Int  deriving (Show,Read)
getStart :: TimeFrame  -> Int
getStart   (TimeFrame start _) = start

getEnd :: TimeFrame  -> Int
getEnd    (TimeFrame _ end) = end

overlap ::  TimeFrame -> TimeFrame -> Bool
overlap x y | not ((getEnd x <  getStart y) || (getEnd y <  getStart x) ) = True
            | otherwise = False
            
startsAfter :: TimeFrame -> TimeFrame -> Bool
startsAfter x y | getStart x > getStart y = True
                | otherwise = False
                
                
mkTimeFrame :: Int -> Int -> TimeFrame  
mkTimeFrame start end
    | start <= end  = TimeFrame start end


safeMkTimeFrame :: Int -> Int -> Maybe TimeFrame  
safeMkTimeFrame start end
    | start <= end = Just (TimeFrame start end)
    | otherwise    = Nothing

In [2]:
data ProcessOper  = Proc         String
                  | Overlap      ProcessOper ProcessOper
                  | StartsAfter  ProcessOper ProcessOper
                  | And          ProcessOper ProcessOper
                  | Or           ProcessOper ProcessOper
                  deriving (Show,Read)
                  
                  
eval :: Subst -> ProcessOper -> Bool

eval s (Overlap p q) = overlap (getTF p s) (getTF q s)
                     where 
                     subst (Proc x) = x 
                     getTF x = find (subst x) 

eval s (StartsAfter p q) = startsAfter (getTF p s) (getTF q s)
                     where 
                     subst (Proc x) = x 
                     getTF x = find (subst x) 

eval s (And p q) = eval s p && eval s q
eval s (Or p q)  = eval s p || eval s q

In [3]:
-- get vars from Process Model

vars :: ProcessOper -> [String]
vars (Proc xs) = [xs]
vars (Overlap p q) = vars p ++ vars q
vars (StartsAfter p q) = vars p ++ vars q
vars (And p q) = vars p ++ vars q
vars (Or p q) = vars p ++ vars q

getvars :: ProcessOper -> [String]
getvars  = sort . nub . vars

-- Visualze process Model

multiplyString :: String -> Int -> String
multiplyString str n = concat (replicate n str)

tf2String :: TimeFrame -> String
tf2String (TimeFrame x y) =    multiplyString " " x  ++   multiplyString "*" (y-x)

strCase :: [(String,TimeFrame)] -> String
strCase  = concatMap ((\x -> tf2String x ++"\n"). snd) 

showCase ::  [(String, TimeFrame)] -> Doc ann
showCase  = pretty . strCase


import Data.List
import Data.List.Split

showListOfProcs ::  [[(String,TimeFrame)]] ->  IO()
showListOfProcs lst = putStrLn $ intercalate "\n ------- \n" (map  strCase  lst)

------------create Asoc Table

In [4]:
import Data.List (permutations)
--import Prettyprinter
import Data.Maybe 

split :: [a]->[(a,a)]
split x = zip [x!! y| y<- evens] [x!! y| y<-odds] 
     where 
       len   =   length x 
       odds  =  [1,3.. len]
       evens =  [0,2.. len]


lst2TimeFrames :: [(Int,Int)] -> [TimeFrame]
lst2TimeFrames  =  f3 . f2 . f1 
    where 
            f1 = map (uncurry safeMkTimeFrame) 
            f2 = filter isJust 
            f3 = map fromJust


myList = [1, 2, 3,4]
permutedLists = permutations myList

aa =  map (lst2TimeFrames . split) permutedLists

aa

[[TimeFrame 1 2,TimeFrame 3 4],[TimeFrame 3 4],[TimeFrame 1 4],[TimeFrame 2 3,TimeFrame 1 4],[TimeFrame 2 4],[TimeFrame 1 3,TimeFrame 2 4],[],[TimeFrame 3 4],[],[],[TimeFrame 2 4],[TimeFrame 2 3],[TimeFrame 2 3],[TimeFrame 1 4,TimeFrame 2 3],[TimeFrame 1 2],[TimeFrame 1 3],[TimeFrame 2 4,TimeFrame 1 3],[],[],[TimeFrame 1 4],[TimeFrame 1 3],[TimeFrame 1 2],[TimeFrame 3 4,TimeFrame 1 2],[]]

In [5]:
-- mkAssocTable :: TimeFrame -> [[string ,TimeFrame ]]

mkAssocTable tf =   filter (\x-> length x == procLstLength tf)  $ map (zip (getvars tf) . lst2TimeFrames. split)  (permLst tf) 
        where 
          procLstLength   = length . getvars
          permLst   tf    = permutations [1..procLstLength tf *2]

findResults :: ProcessOper -> [[(String,TimeFrame)]] 
findResults p = filter (`eval` p ) (mkAssocTable p)

--mkAssocTable p1
--getvars p1

In [6]:
assocTableCase = [("procesA",TimeFrame 1 2),("procesB",TimeFrame 3 4),("procesC",TimeFrame 5 6)]

p1 :: ProcessOper
p1 = And ( StartsAfter (Proc "procesB") (Proc "procesA")) 
         ( Overlap     (Proc "procesA") (Proc "procesC")) 
         
         
showCase assocTableCase             
eval assocTableCase p1

 *
   *
     *

False

In [7]:
--- Testing 

p1 :: ProcessOper
p1 = And ( StartsAfter (Proc "procesA") (Proc "procesB")) 
         ( Overlap     (Proc "procesB") (Proc "procesC"))

p2 :: ProcessOper
p2 = StartsAfter (Proc "procesA") (Proc "procesC")

p3 :: ProcessOper
p3 = Or p1 p2
    

(showListOfProcs . take 5 .findResults) p3



   *
  ***
 *****

 ------- 
  ***
   *
 *****

 ------- 
  *
    *
 *****

 ------- 
   **
  **
 *****

 ------- 
  **
   **
 *****

-----------

In [8]:
import Data.Text.Prettyprint.Doc

data TimeFrame = TimeFrame Int Int deriving (Show, Read)

instance Pretty TimeFrame where
    pretty (TimeFrame start end) = pretty "TF(" <+> pretty start <+>  pretty end <+> pretty ")"


t =  TimeFrame 10 20
pretty [("procesA",TimeFrame 3 4),("procesB",TimeFrame 2 5),("procesC",TimeFrame 1 6)]

[(procesA, TF( 3 4 )), (procesB, TF( 2 5 )), (procesC, TF( 1 6 ))]